# **Нарезка изображений под YOLOv8 для до-обучения**

# **Сервисные функции**

In [ ]:
import time

# @title Контекстный менеджер для измерения времени операций
class timex:
    def __enter__(self):
        # Фиксация времени старта процесса
        self.t = time.time()
        return self

    def __exit__(self, type, value, traceback):
        # Вывод времени работы
        # Расчет времени выполнения
        result_time = time.time()-self.t
        hour = int(result_time//3600)
        min = int(result_time//60)-hour*60
        sec = int(round(result_time%60))
        msec = round(1000*result_time%60)

        if hour > 0:
          print('\nВремя обработки: ' + str(hour)+' час. ' + str(min)+' мин.')
        elif min > 0:
          print('\nВремя обработки: ' + str(min)+' мин. ' + str(sec)+' сек.')
        elif sec > 0:
          print('\nВремя обработки: ' + str(sec)+' сек.')
        else:
          print('\nВремя обработки: ' + str(msec)+' мс.')


In [ ]:
import gdown

# @title Функция загрузки кадра из видео в Google Colab
def load_video(video_url, video_name):
    # Стартовая отметка времени
    t_0 = time.time()

    # Загружаем видео в Colab
    gdown.download(video_url, None, quiet=True)

    video_path = f"/content/{video_name}"

    # Расчет времени выполнения
    t_f = time.time()-t_0

    # Расчет времени обработи и приведение его к легко читаемому виду
    time_text = str(int(round(t_f//60,0)))+' мин '+str(int(round(t_f%60,0)))+' сек.'

    # Подведение итогов обработки
    print('Загрузка видеофайла: '+ video_name + '\nсделана за: ' + time_text)

    return video_path


In [ ]:
from google.colab.patches import cv2_imshow
import cv2
from pathlib import Path

# @title Функция извлечения кадров из видео по указанным временным меткам
def extract_frames_from_video(video_path, time_list, path_output):
    # Стартовая отметка времени
    t_0 = time.time()

    # Открываем видеофайл
    cap = cv2.VideoCapture(video_path)

    # Проверяем, успешно ли открыт файл
    if not cap.isOpened():
        print(f"Не удалось открыть видеофайл: {video_path}")
        return None

    frames = []  # Список для хранения извлеченных кадров

    # Обрабатываем каждую временную метку
    for time_point in time_list:
        # Рассчитываем время в кадрах
        target_frame_time = int((time_point[0] * 60 + time_point[1]) * cap.get(cv2.CAP_PROP_FPS))

        # Устанавливаем позицию в видео на нужный кадр
        cap.set(cv2.CAP_PROP_POS_FRAMES, target_frame_time)

        # Считываем кадр
        ret, frame = cap.read()

        # Проверяем, успешно ли считан кадр
        if not ret:
            print(f"Не удалось считать кадр для времени {time_point[0]} минут {time_point[1]} секунд.")
            continue

        # Добавляем кадр в список
        frames.append(frame)

    # Отключаем видеопоток
    cap.release()

    # Расчет времени выполнения
    t_f = time.time() - t_0

    # Расчет времени обработи и приведение его к легко читаемому виду
    time_text = str(int(round(t_f // 60, 0))) + ' мин ' + str(int(round(t_f % 60, 0))) + ' сек.'

    # Подведение итогов обработки
    print(f'Из видеофайла: {video_path}\nизвлечены кадры для времен: {time_list}\nза: {time_text}\n')

    # Создаем папку Image, если она не существует
    Path(path_output).mkdir(parents=True, exist_ok=True)

    # Сохраняем извлеченные кадры в папке Image
    for i, frame in enumerate(frames):
        # Формируем имя файла на основе времени и сохраняем в папке Image
        file_name = f"frame_{time_list[i][0]:02d}min_{time_list[i][1]:02d}sec.png"
        file_path = Path(path_output) / file_name
        # cv2.imwrite(str(file_path), cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        cv2.imwrite(str(file_path), frame)  # Убрали cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Выводим информацию о сохраненном файле
        print(f"Сохранен файл: {file_path}")
        print(f"Разрешение файла: {frame.shape[1]}x{frame.shape[0]} пикселей\n")

    return frames


In [ ]:
# @title Функция изменения ширины кадра (высота пропорционально)
def show_resized_images(frames, new_width):
    """
    Отображает изображения с установленной новой шириной, сохраняя пропорции.

    Параметры:
    - frames: List[numpy.ndarray], список входных изображений.
    - new_width: int, новая ширина изображения.
    """
    for i, frame in enumerate(frames):
        aspect_ratio = frame.shape[1] / frame.shape[0]
        new_height = int(new_width / aspect_ratio)
        resized_image = cv2.resize(frame, (new_width, new_height))
        cv2_imshow(resized_image)
        print()


In [ ]:
import shutil
import zipfile

# @title Функция архивации содержимого папки и сохранения архива по указанному пути
def zip_folder(folder_path, zip_path):
    """
    Архивирует содержимое папки и сохраняет архив по указанному пути.

    Параметры:
    - folder_path: str, путь к архивируемой папке.
    - zip_path: str, путь, по которому сохраняется zip-архив.
    """
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                arc_name = os.path.relpath(file_path, folder_path)
                zipf.write(file_path, arcname=arc_name)


In [ ]:
import os
import cv2
import matplotlib.pyplot as plt

# @title Функция (1) нарезки изображения из указанной папки на фрагменты с указанием размера фрагмента и сохранение их
def crop_and_save_images_by_fragment_size_enlarged(input_folder, output_folder, fragment_size=640):
    """
    Нарезает изображения из указанной папки на фрагменты размером fragment_size x fragment_size и сохраняет их.
    КАЖДЫЙ ФРАГМЕНТ ОТДЕЛЬНО И УВЕЛИЧЕННЫЙ ДЛЯ ЛУЧШЕГО ПРОСМОТРА

    Параметры:
    - input_folder: str, путь к папке с изображениями.
    - output_folder: str, путь к папке для сохранения результата.
    - fragment_size: int, размер фрагмента.
    """
    # Создаем каталог для сохранения результата
    if not os.path.isdir(output_folder):
        os.mkdir(output_folder)

    # Проходим по всем файлам в папке
    for file_name in os.listdir(input_folder):
        # Проверяем, что файл является изображением
        if file_name.endswith(('.png', '.jpg', '.jpeg')):
            # Полный путь к файлу
            file_path = os.path.join(input_folder, file_name)

            # Загрузка изображения
            img = cv2.imread(file_path, cv2.IMREAD_COLOR)

            # Извлечение имени файла без расширения
            base_name, _ = os.path.splitext(file_name)

            # Проверяем, нужно ли обрезать изображение по высоте
            new_height = (img.shape[0] // fragment_size) * fragment_size
            if img.shape[0] != new_height:
                img = img[img.shape[0] - new_height:, :]

            # Создаем каталог для сохранения фрагментов
            cropped_subfolder = os.path.join(output_folder, base_name + '_cropped')
            os.makedirs(cropped_subfolder, exist_ok=True)

            # Сохраняем оригинальное изображение в папке с нарезанными фрагментами
            original_file_path = os.path.join(cropped_subfolder, f"{base_name}_original.png")
            cv2.imwrite(original_file_path, img)

            # Выводим оригинал на печать
            plt.figure(figsize=(8, 8))
            plt.imshow(img[:, :, ::-1])
            plt.title(f"Original - {base_name}")
            plt.axis('off')
            plt.show()

            # Размеры изображения после обрезки
            height, width, _ = img.shape

            # Вычисляем, на сколько фрагментов можно разделить по ширине и высоте
            num_width_fragments = width // fragment_size
            num_height_fragments = height // fragment_size

            # Выводим изображения для проверки
            for i in range(num_height_fragments):
                for j in range(num_width_fragments):
                    sub_img = img[i * fragment_size:(i + 1) * fragment_size, j * fragment_size:(j + 1) * fragment_size, ::-1]
                    sub_name = f"{fragment_size}x{fragment_size}_part_{i * num_width_fragments + j + 1}"

                    plt.figure(figsize=(8, 8))
                    plt.imshow(sub_img)
                    plt.title(sub_name)
                    plt.axis('off')
                    plt.show()

                    # Сохраняем фрагмент в папку
                    sub_file_name = f"{base_name}_{sub_name}.png"
                    sub_file_path = os.path.join(cropped_subfolder, sub_file_name)
                    cv2.imwrite(sub_file_path, sub_img[:, :, ::-1])

                    # Закрываем фигуру после использования
                    # plt.close()


In [ ]:
import os
import cv2
import matplotlib.pyplot as plt

# @title Функция (2) нарезки изображения из указанной папки на фрагменты с указанием размера фрагмента и сохранение их
def crop_and_save_images_by_fragment_size(input_folder,
                                          output_folder,
                                          fragment_size=640,
                                          crop_height_from="t",
                                          crop_width_from="l"):
    """
    Нарезает изображения из указанной папки на фрагменты размером fragment_size x fragment_size и сохраняет их.
    ВСЕ ФРАГМЕНТЫ ВМЕСТЕ НА ОДНОМ ПОЛОТНЕ

    "t" для "top"
    "b" для "bottom"
    "l" для "left"
    "r" для "right"

    Параметры:
    - input_folder: str, путь к папке с изображениями.
    - output_folder: str, путь к папке для сохранения результата.
    - fragment_size: int, размер фрагмента.
    """
    # Создаем каталог для сохранения результата
    if not os.path.isdir(output_folder):
        os.mkdir(output_folder)

    # Проходим по всем файлам в папке
    for file_name in os.listdir(input_folder):
        # Проверяем, что файл является изображением
        if file_name.endswith(('.png', '.jpg', '.jpeg')):
            # Полный путь к файлу
            file_path = os.path.join(input_folder, file_name)

            # Загрузка изображения
            img = cv2.imread(file_path, cv2.IMREAD_COLOR)

            # Извлечение имени файла без расширения
            base_name, _ = os.path.splitext(file_name)

            # Проверяем, нужно ли обрезать изображение по высоте
            new_height = (img.shape[0] // fragment_size) * fragment_size
            if img.shape[0] != new_height:
                if crop_height_from == "t":
                    img = img[img.shape[0] - new_height:, :]
                elif crop_height_from == "b":
                    img = img[:new_height, :]

            # Проверяем, нужно ли обрезать изображение по ширине
            new_width = (img.shape[1] // fragment_size) * fragment_size
            if img.shape[1] != new_width:
                if crop_width_from == "l":
                    img = img[:, img.shape[1] - new_width:]
                elif crop_width_from == "r":
                    img = img[:, :new_width]

            # Создаем каталог для сохранения фрагментов
            cropped_subfolder = os.path.join(output_folder, base_name + '_cropped')
            os.makedirs(cropped_subfolder, exist_ok=True)

            # Сохраняем оригинальное изображение в папке с нарезанными фрагментами
            original_file_path = os.path.join(cropped_subfolder, f"{base_name}_original_H_{crop_height_from}_W_{crop_width_from}.png")
            cv2.imwrite(original_file_path, img)

            # Выводим оригинал на печать
            plt.figure(figsize=(20, 12))
            plt.imshow(img[:, :, ::-1])
            plt.title(f"Original - {base_name}")
            plt.axis('off')
            plt.show()

            # Размеры изображения после обрезки
            height, width, _ = img.shape

            # Вычисляем, на сколько фрагментов можно разделить по ширине и высоте
            num_width_fragments = width // fragment_size
            num_height_fragments = height // fragment_size

            # Выводим изображения для проверки
            plt.figure(figsize=(20, 10))

            for i in range(num_height_fragments):
                for j in range(num_width_fragments):
                    sub_img = img[i * fragment_size:(i + 1) * fragment_size, j * fragment_size:(j + 1) * fragment_size, ::-1]
                    sub_name = f"{fragment_size}x{fragment_size}_part_{i * num_width_fragments + j + 1}_H_{crop_height_from}_W_{crop_width_from}"

                    # Размещаем фрагмент изображения на соответствующем подграфике
                    ax = plt.subplot(num_height_fragments, num_width_fragments, i * num_width_fragments + j + 1)
                    ax.get_xaxis().set_visible(False)
                    ax.get_yaxis().set_visible(False)
                    ax.set_title(sub_name)
                    plt.imshow(sub_img)

                    # Сохраняем фрагмент в папку
                    sub_file_name = f"{base_name}_{sub_name}.png"
                    sub_file_path = os.path.join(cropped_subfolder, sub_file_name)
                    cv2.imwrite(sub_file_path, sub_img[:, :, ::-1])


In [5]:
import gdown
import zipfile
import os
import matplotlib.pyplot as plt
from PIL import Image

# @title Функция загрузки zip с изображениями и отображение их на полотне
def download_and_display_images(url, num_rows=3, num_cols=2, figsize=(10, 10)):
    """
    Загружает изображения из zip-файла по указанной ссылке и отображает их на полотне.

    Параметры:
    - url: str, ссылка на zip-файл с изображениями.
    - num_rows: int, количество строк на полотне.
    - num_cols: int, количество столбцов на полотне.
    - figsize: tuple, размер полотна (ширина, высота).

    Пример использования:
    download_and_display_images('ссылка_на_ваш_файл.zip', num_rows=3, num_cols=2, figsize=(10, 10))
    """
    # Загружаем файл zip
    output_zip = 'downloaded_images.zip'
    gdown.download(url, output_zip, quiet=False)

    # Удаляем все файлы в output_folder
    output_folder = 'extracted_images'
    if os.path.exists(output_folder):
        for file_name in os.listdir(output_folder):
            file_path = os.path.join(output_folder, file_name)
            os.remove(file_path)
    else:
        os.makedirs(output_folder)

    # Разархивируем файл
    output_folder = 'extracted_images'
    with zipfile.ZipFile(output_zip, 'r') as zip_ref:
        zip_ref.extractall(output_folder)

    # Получаем список файлов изображений в папке, отсортированный по именам
    image_files = sorted([file for file in os.listdir(output_folder) if file.endswith(('.png', '.jpg', '.jpeg'))])

    # Создаем полотно для изображений
    plt.figure(figsize=figsize)

    # Отображаем изображения на полотне
    for i, image_file in enumerate(image_files, start=1):
        img_path = os.path.join(output_folder, image_file)
        img = Image.open(img_path)
        plt.subplot(num_rows, num_cols, i)
        plt.imshow(img)
        # plt.title(f"Image {i}")
        plt.title(f"{image_file}")
        plt.axis('off')

    plt.tight_layout()
    plt.show()


# **Загрузка видео в Google Colab**

In [ ]:
# @title Загрузка видео-файла DJI_0002 (1).MP4...
# video_url = 'https://drive.google.com/uc?id=1Kk5fx8C4nCe9Drxl-wTMP5MfjU4H1F0F'  # Мой аккаунт
video_url = 'https://drive.google.com/uc?id=1CLrFoJtqHSZhCimWFb4bV9HY-JLelBHr'  # Общий аккаунт

video_name = 'DJI_0002 (1).MP4'

# Загрузка видео
video_path_1 = load_video(video_url, video_name)


Загрузка видеофайла: DJI_0002 (1).MP4
сделана за: 0 мин 46 сек.


In [ ]:
# @title Загрузка видео-файла DJI_0001 (12).MP4...
# video_url = 'https://drive.google.com/uc?id=16X2wW08AJ417tY9Axa9wALxnlSOHvkIM'  # Мой аккаунт
video_url = 'https://drive.google.com/uc?id=1rSBDMx8eKpdsQOOwfdgeaHzb8WpOkex2'  # Общий аккаунт

video_name = 'DJI_0001 (12).MP4'

# Загрузка видео
video_path_2 = load_video(video_url, video_name)


Загрузка видеофайла: DJI_0001 (12).MP4
сделана за: 0 мин 39 сек.


# **Извлечение кадров из видео-файла**

In [ ]:
# @title Извлечение кадров из списка временных меток в формате (минуты, секунды) из видео DJI_0002 (1).MP4...

time_list_1 = [(0, 30), (1, 0), (2, 15)]  # Список временных меток в формате (минуты, секунды)
path_output_1 = '/content/ImageFromVideo_1'

frames_1 = extract_frames_from_video(video_path_1, time_list_1, path_output_1)

# Вывод изображения
if frames_1 is not None:
    show_resized_images(frames_1, new_width=1500)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# @title Извлечение кадров из списка временных меток в формате (минуты, секунды) из видео DJI_0001 (12).MP4...

time_list_2 = [(0, 30), (1, 0), (2, 15)]  # Список временных меток в формате (минуты, секунды)
path_output_2 = '/content/ImageFromVideo_2'

frames_2 = extract_frames_from_video(video_path_2, time_list_2, path_output_2)

# Вывод изображения
if frames_2 is not None:
    show_resized_images(frames_2, new_width=1500)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# @title Архивация содержимого папок и сохранение архивов по указанному пути
folder_to_zip = '/content/ImageFromVideo_1'
zip_output_path = '/content/ImageFromVideo_1.zip'

zip_folder(folder_to_zip, zip_output_path)

folder_to_zip = '/content/ImageFromVideo_2'
zip_output_path = '/content/ImageFromVideo_2.zip'

zip_folder(folder_to_zip, zip_output_path)


In [ ]:
# @title Нарезка изображения из указанной папки на фрагменты 640x640 (cropped top)
input_folder = '/content/ImageFromVideo_1/'
output_folder = '/content/CroppedImagesFromVideo_1/'

crop_and_save_images_by_fragment_size(input_folder, output_folder, fragment_size=640, crop_height_from="t")


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# @title Нарезка изображения из указанной папки на фрагменты 640x640  (cropped bottom)
input_folder = '/content/ImageFromVideo_1/'
output_folder = '/content/CroppedImagesFromVideo_1/'

crop_and_save_images_by_fragment_size(input_folder, output_folder, fragment_size=640, crop_height_from="b")


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# @title Нарезка изображения из указанной папки на фрагменты 1280x1280 (cropped top)
input_folder = '/content/ImageFromVideo_1/'
output_folder = '/content/CroppedImagesFromVideo_1/'

crop_and_save_images_by_fragment_size(input_folder, output_folder, fragment_size=1280, crop_height_from="t")


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# @title Нарезка изображения из указанной папки на фрагменты 1280x1280 (cropped bottom)
input_folder = '/content/ImageFromVideo_1/'
output_folder = '/content/CroppedImagesFromVideo_1/'

crop_and_save_images_by_fragment_size(input_folder, output_folder, fragment_size=1280, crop_height_from="b")


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# @title Архивация содержимого папки с нарезками и сохранение архива
folder_to_zip = '/content/CroppedImagesFromVideo_1'
zip_output_path = '/content/CroppedImagesFromVideo_1.zip'

zip_folder(folder_to_zip, zip_output_path)


In [ ]:
# @title Нарезка изображения из указанной папки на фрагменты 640x640 (cropped top)
input_folder = '/content/ImageFromVideo_2/'
output_folder = '/content/CroppedImagesFromVideo_2/'

crop_and_save_images_by_fragment_size(input_folder, output_folder, fragment_size=640, crop_height_from="t")


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# @title Нарезка изображения из указанной папки на фрагменты 640x640 (cropped bottom)
input_folder = '/content/ImageFromVideo_2/'
output_folder = '/content/CroppedImagesFromVideo_2/'

crop_and_save_images_by_fragment_size(input_folder, output_folder, fragment_size=640, crop_height_from="b")


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# @title Нарезка изображения из указанной папки на фрагменты 1280x1280 (cropped top)
input_folder = '/content/ImageFromVideo_2/'
output_folder = '/content/CroppedImagesFromVideo_2/'

crop_and_save_images_by_fragment_size(input_folder, output_folder, fragment_size=1280, crop_height_from="t")


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# @title Нарезка изображения из указанной папки на фрагменты 1280x1280 (cropped bottom)
input_folder = '/content/ImageFromVideo_2/'
output_folder = '/content/CroppedImagesFromVideo_2/'

crop_and_save_images_by_fragment_size(input_folder, output_folder, fragment_size=1280, crop_height_from="b")


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# @title Архивация содержимого папки с нарезками и сохранение архива
folder_to_zip = '/content/CroppedImagesFromVideo_2'
zip_output_path = '/content/CroppedImagesFromVideo_2.zip'

zip_folder(folder_to_zip, zip_output_path)


# **Примеры нарезки изображений**

## **Пример 1: Автокран**

In [2]:
# @title Загрузка zip с изображениями и отображение примера #1: Автокран
zip_url = 'https://drive.google.com/uc?id=1arMzErdbdQB_b-5w9MHDIFY9REvmv6vm'

download_and_display_images(zip_url, num_rows=3, num_cols=2, figsize=(12, 12))


Output hidden; open in https://colab.research.google.com to view.

## **Пример 2: Малое строение**

In [6]:
# @title Загрузка zip с изображениями и отображение примера #2: Малое строение
zip_url = 'https://drive.google.com/uc?id=1vOjaq9iVDZDPc__F-2KL9okHKLZ6cUyr'

download_and_display_images(zip_url, num_rows=3, num_cols=2, figsize=(12, 12))


Output hidden; open in https://colab.research.google.com to view.

# **Вопросы**

## **Подсчёт количества кадров для обработки:**

Берём 2 видео из последней по времени папки 182 27.04.2023. В этих видео есть все интересующие нас объекты.

В видео_1: 220 сек., в видео_2: 290 сек. Итого: 510 сек.

Если извлекать кадр каждые 10 сек., то получится 51 кадр с разрешением 3840х2160 пикселей.

Перед подачей изображения в YOLOv8 все изображения нужно привести к единому разрешению 640х640.

**Правильно? Типа как контролируемая предобработка изображений перед подачей их в YOLOv8 для обучения. Чтобы не полагаться на саму YOLOv8.**

За счёт обрезки кадра либо сверху либо снизу для того, чтобы кадры были кратны 640х640, может получиться 2244 изображений. (Т.е. на 1 извлечённый кадр получим изображения с разрешениями 640х640 (36 шт), 1280х1280 (6 шт.), 3840х1920 (2 шт.))

Смещение кадров по вертикали получилось естественным образом за счёт обрезки кадра. Если также сделать смещение по горизонтали, скажем в пол-кадра 640 / 2 = 320 пикселей, сначала слева, затем нарезать. Потом смещение справа, затем нарезать, то получится ещё больше. **Это правильный подход? Сделать ещё смещение по горизонтали слева и справа как параметр?**

Эти смещения по горизонтали слева и справа, а также обрезание кадра для кратности 640 сверху и снизу можно делать по отдельности и одновременно.

Таким образом из этих 2-х видео можно получить достаточно изображений разного масштаба, где практически все нужные нам объекты будут если не по центру изображения, то по крайней мере целиком находиться в кадре.

Далее нужно будет их просмотреть, удалить ненужные фрагменты, в которых вообще нет объектов интереса, и затем рассортировать их по папкам (классам)

**Объекты интереса:**

> **Object Detection**

>> Корпуса

>> Строительная площадка (просто вся площадка целиком?)

>> Строительная техника

> **Segmention**

>> Объекты благоустройства (газон, лужайка, дороги)

>> Насыпи

>> Ямы с водой

**Tower Crane - Башенный кран**

**Вопрос:** у башенного крана есть стрела.

Дмитрий показывал как YOLOv8 их обнаруживает. У него она обнаруживает саму башню без стрелы.

Это правильно?

С одной стороны, если бы YOLOv8 обнаруживала башенный кран полностью со стрелой, то рамка обнаружения может быть большой, если он расположен к нам боком. И вроде как лучше тогда его сегментировать, а не детектировать.

А с другой стороны, не очень удобно сегментировать остальную строительную технику из-за сложности её контуров.

**Т.е. обнаруживать в башенном кране только башню это оптимально???**

Но тогда вопрос по разметке и обучению.

**У башенного крана размечать только башню?**

Ещё такой вопрос:

Если какой-либо объект не полностью присутствует на изображении (фрагменте от полного изображения), скажем тот же башенный кран, то его не нужно размечать???

Или башенный кран можно, а вот другие не желательно? например, половина экскаватора или грейдера?

Если 50% от целого, то не стоит размечать, а если 80-90%???

Какой критерий? Может быть совет...

Такой же вопрос как предыдущий, но по ямам с водой, насыпям...

Размечать ли их на фрагментах, если они не полностью присутствуют на этом фрагменте?

Может быть размечать их, если они на фрагменте примерно 70-80% от целого, на глаз примерно.

А если меньше, то уже не размечать.

## **Вопрос по количеству изображений каждого класса**

ChatGPT выдала минимальное число изображений на один класс это 100 шт.

Понятное дело, что "кашу маслом не испортишь" :-) и чем больше, тем лучше...

но всё же на какое минимальное число можно рассчитывать и уже смело пытаться обучать, ожидая приемлемое качество разпознавания?

## **Вопрос по инструментам разметки**

Каким инструментом делать разметку?

Заказчик разрешил использовать Roboflow?